In [1]:
# Importing the required libraries
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import Tracker
import os
import matplotlib.pyplot as plt
import time

In [2]:
# Importing and installing the required files

# Load YOLO model
model = YOLO('yolov8s.pt')

# Define the class list for object detection
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 
              'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 
              'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 
              'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 
              'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 
              'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 
              'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 
              'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

tracker = Tracker()  # Initialize tracker for objects
count = 0

In [3]:
# Load the video file
cap = cv2.VideoCapture('video.mp4')

# Initialize dictionaries for tracking vehicles passing the red and blue lines
down = {}
up = {}
counter_down = []
counter_up = []

# Define lines for tracking vehicle movement
red_line_y = 198
blue_line_y = 268
offset = 6

# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

# Set the codec and initialize video writer for output in MP4 format
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (1020, 500))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame, (1020, 500))

    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    detections = []

    # Collect the bounding boxes for cars and trucks
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c or 'truck' in c:  # Detect only cars and trucks
            detections.append([x1, y1, x2 - x1, y2 - y1])

    # Update tracker with current frame's detections
    bbox_id = tracker.update(detections)

    # Process each detected vehicle
    for bbox in bbox_id:
        x3, y3, w, h, id = bbox
        x4 = x3 + w
        y4 = y3 + h
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)

        # Detect vehicle crossing the red line (downward direction)
        if red_line_y - offset < cy < red_line_y + offset:
            down[id] = time.time()
        if id in down:
            if blue_line_y - offset < cy < blue_line_y + offset:
                elapsed_time = time.time() - down[id]
                if counter_down.count(id) == 0:
                    counter_down.append(id)
                    distance = 10  # Assume distance between red and blue line is 10 meters
                    speed_ms = distance / elapsed_time
                    speed_kh = speed_ms * 3.6  # Convert m/s to km/h
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(speed_kh)) + ' Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

        # Detect vehicle crossing the blue line (upward direction)
        if blue_line_y - offset < cy < blue_line_y + offset:
            up[id] = time.time()
        if id in up:
            if red_line_y - offset < cy < red_line_y + offset:
                elapsed1_time = time.time() - up[id]
                if counter_up.count(id) == 0:
                    counter_up.append(id)
                    distance1 = 10  # Assume distance between red and blue line is 10 meters
                    speed_ms1 = distance1 / elapsed1_time
                    speed_kh1 = speed_ms1 * 3.6  # Convert m/s to km/h
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(speed_kh1)) + ' Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

    # Draw lines and speed info on frame
    text_color = (0, 0, 0)
    yellow_color = (0, 255, 255)
    red_color = (0, 0, 255)
    blue_color = (255, 0, 0)

    cv2.rectangle(frame, (0, 0), (250, 90), yellow_color, -1)
    cv2.line(frame, (172, red_line_y), (774, red_line_y), red_color, 2)
    cv2.putText(frame, 'Red Line', (172, red_line_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.line(frame, (8, blue_line_y), (927, blue_line_y), blue_color, 2)
    cv2.putText(frame, 'Blue Line', (8, blue_line_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, 'Going Down - ' + str(len(counter_down)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, 'Going Up - ' + str(len(counter_up)), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    # Save frame to file and write to output video
    frame_filename = f'detected_frames/frame_{count}.jpg'
    cv2.imwrite(frame_filename, frame)
    out.write(frame)

    # Display the frame using matplotlib
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title("Frame")
    plt.axis("off")
    plt.show()

In [4]:
# Release resources
cap.release()
out.release()
plt.close('all')